In [ ]:
from PIL import ImageGrab
import keyboard
import time
import sys
import json

# OCR用
import os
from PIL import Image
import pyocr
import pyocr.builders

#曲名取得用
import pandas as pd
import difflib

### settings.jsonファイル読み込み
settings_file = open("settings.json","r", encoding='utf-8')
settings_data = json.load(settings_file)

### OCR準備
# 1.インストール済みのTesseractのパスを通す
path_tesseract = "C:\\Program Files\\Tesseract-OCR"
if path_tesseract not in os.environ["PATH"].split(os.pathsep):
    os.environ["PATH"] += os.pathsep + path_tesseract

# 2.OCRエンジンの取得
tools = pyocr.get_available_tools()
tool = tools[0]

### 曲名取得
#tsvファイル読み込み
df = pd.read_table(settings_data["music_list_filename"])
#ratio列を追加
df["ratio"] = 0.0


def capture():
    # full screen
    ImageGrab.grab().save(settings_data["capture_filename"])
    print("a")

    # 指定した領域内をクリッピング
    clip= ImageGrab.grab(
        bbox=(settings_data["left-top"],
              settings_data["left-bottom"],
              settings_data["right-top"],
              settings_data["right-bottom"])
    )
    print("b")
    clip.save(settings_data["capture_clip_filename"])
    print("c")

    
    #画像を読みやすいように加工
    img = clip
    img = img.convert('RGB')
    size = img.size
    img2 = Image.new('RGB',size)

    #二値化
    border=140
    border2=140
    for x in range(size[0]):
        for y in range(size[1]):
            r,g,b=img.getpixel((x,y))
            if r > border or g > border or b > border:
                r = 255
                g = 255
                b = 255
            if r < border2 or g < border2 or b < border2:
                r = 0
                g = 0
                b = 0
            img2.putpixel((x,y),(r,g,b))

    #白黒反転
    img3 = Image.new('RGB',size)
    for x in range(size[0]):
        for y in range(size[1]):
            r,g,b=img2.getpixel((x,y))
            if r == 0 and g == 0 and b == 0:
                r = 255
                g = 255
                b = 255
            else:
                r = 0
                g = 0
                b = 0
            img3.putpixel((x,y),(r,g,b))
 
    print("d")
    img3.save("PIL_capture_clip_2.png")
    print("e")

def ocr():
    # 3.原稿画像の読み込み
    #img_org = Image.open(settings_data["capture_clip_filename"])
    img_org = Image.open("PIL_capture_clip_2.png")

    # 4.ＯＣＲ実行
    #builder = pyocr.builders.TextBuilder()
    builder = pyocr.builders.TextBuilder(tesseract_layout=7)
    return tool.image_to_string(img_org, lang="jpn", builder=builder)

def get_title(ocr_title):
    #1行ずつratio算出
    for index, row in df.iterrows():
        ratio = difflib.SequenceMatcher(None, row.Title, ocr_title).ratio()
        row.ratio = ratio
        df.at[index, 'ratio'] = ratio

    #一番近いタイトルをSeriesで取得
    s = df.loc[df['ratio'].idxmax()]
    return s.Title   

def write_file_title(write_str):
    f = open(settings_data["title_text_filename"], 'w', encoding='utf-8')
    f.write(write_str)
    f.close()
    

def loop():
    while True:
        if keyboard.is_pressed("p"):
            #if keyboard.read_event().event_type == keyboard.KEY_DOWN:
            print("You pressed p")

            #画面キャプチャ
            capture()
            
            #OCR
            ocr_title = ocr()
            print("ocr   title:", ocr_title)
            
            #曲名一覧から、一番近いタイトルを取得
            music_title = get_title(ocr_title)
            print("match title:", music_title)
            
            #タイトルをテキストファイルに出力
            write_file_title(music_title)

        if keyboard.is_pressed("q"):
            print("You pressed q")
            break
            
        time.sleep(0.1)

loop()

You pressed p
a
b
c
d
e
ocr   title: 不沈艦CANDY
match title: 不沈艦CANDY
You pressed p
a
b
c
d
e
ocr   title: 黒髪乱れし修比となりて
match title: 黒髪乱れし修羅となりて
You pressed p
a
b
c
d
e
ocr   title: フェティイッジュベイパー て 腔の汗回新ガール て
match title: フェティッシュペイパー ～脇の汗回転ガール～
You pressed p
a
b
c
d
e
ocr   title: 5O+h Memori Songs The BEMANI Hstoru-
match title: 50th Memorial Songs -The BEMANI History-
You pressed p
a
b
c
d
e
ocr   title: 刃図宮
match title: 刃図羅
You pressed p
a
b
c
d
e
ocr   title: 知熱Beach Side Bunnu
match title: 灼熱Beach Side Bunny
You pressed p
a
b
c
d
e
ocr   title: 5Oth Memorial Songs -Flaqship rnedleu-
match title: 50th Memorial Songs -Flagship medley-
You pressed p
a
b
c
d
e
ocr   title: AA
match title: AA
You pressed p
a
b
c
d
e
ocr   title: Despair of ELFERIA
match title: Despair of ELFERIA
You pressed p
a
b
c
d
e
ocr   title: Bad Maniacs
match title: Bad Maniacs
You pressed p
a
b
c
d
e
ocr   title: Cione (Ruu支 Remix)
match title: Clione (Ryu☆ Remix)
You pressed p
a
b
c
d
e
ocr   title: chaos eater ID